# Position Based Dynamics

We will follow along with this presentation of PBD: https://matthias-research.github.io/pages/publications/posBasedDyn.pdf

The basic equations for position based dynamics converted to Geometric Algebra are very similar.  

The differential operators used in GA differ from those of Vector Calculus, so it will look a little different in places.

The gradient operator in GA using index notation is $e_i \partial_i$

To formulate a Constraint Projection solver we start with a constraint equation, $C(p)$, that take positions as arguments and will equal zero when it is satisfied.

Taylor expansion of a scalar valued function using the differential

$C(p + \Delta p) \approx C(p) + \Delta p_i \partial_i C(p) = 0$ &emsp; &emsp; (1)

We want the gradient operator to appear.  We use the fact that, for our euclidean basis,

$e_i \cdot e_j = \delta_{ij}$ &emsp; &emsp; (2) 

Note: if we were using a non-euclidean metric, this would need to be modified.

$C(p) + \Delta p_i e_i \cdot e_i \partial_i C(p) = C(p) + \Delta p \cdot \nabla C(p) = 0$ &emsp; &emsp; (3)

Ok, we've recovered the first Position Based Dynamics formula, while setting ourselves up for extension to non-eucledian metrics.

$\Delta p = \lambda \nabla C(p)$   &emsp; &emsp; (4)
    
Solving for $\Delta p$ in terms of known values gives

$\Delta p = \frac{-C(p)}{\nabla C(p) \cdot \nabla C(p)} \nabla C(p)$ &emsp; &emsp; (5)


## G(n,0,0) the algebra of directions 

The most familiar algebra for those familiar with Position Based Dynamics will be G(n,0,0). This means all the basis vectors square to 1.  This is the metric of euclidean space.

The gradient operator is slightly different.  Rather than organizing the resulting partial derivatives in a column vector, partials are attached to basis vectors.

$\nabla f(p) = e_1 \partial_x f(p) + e_2 \partial_y f(p) + e_3 \partial_z f(p)$

rather than

$\nabla f(p) = \begin{bmatrix}
\partial_x f(p) \\ 
\partial_y f(p)\\ 
\partial_z f(p)
\end{bmatrix}$

we will use (1,2,3) rather than (x,y,z) to specify partial derivatives from now on.

If you wish you can treat the gradient operator as a 1-vector in GA just as you can in Linear Algebra.

## G(n,0,1) the algebra of planes.  aka PGA

Our first non-euclidean metric.  This algebra has n basis vectors that square to 1 and a single basis vector that squares to 0.  We will assume n = 3 for now.
The basis vector that squares to zero is similar to the w component of homogeneous vectors you may be familiar with.
Effectively this lets us efficiently model projective geometry.  
We choose $e_{n}$ as our degenerate basis vector.  Since Julia is our implementation language we stick with 1-based indexing and choose to place the degenerate basis element as the last element of a 1-vector to mirror the use of w in a Vector Algebra setting.  i.e. $e_1*e_1 == 1$, $e_2*e_2 == 1$, $e_3*e_3 == 1$, and $e_4*e_4 == 0$ for 3D projective geometry.

One more thing we need to do for maximum elegance and power is to represent our points as 3-vectors, which is visualized as the intersection of 3 planes.  This can take a little while to get used to.  For more information about PGA please check out https://bivector.net

This unreasonably effective algebra has many advantages.  Almost trivial representation/construction of Dual Quaternions and the ability to represent objects at infinity being it's two main super-powers.

### PBD using PGA

A few things we need to consider now that we have a degenerate metric and trivector based positions.

We need to revisit our foundational equations and see what happens with these changes.

The degenerate metric prevents us from using the previous trick we used to get a gradient operator (2), as it's not true for $e_4$.

Instead we multiply (2) by the pseudoscalar $\textit{I} = e_{1234}$.  
We can factor the pseudoscalar into the 1-vectors for the gradient and the 3-vectors for points.  Factorization is via the wedge product.

$e_{1234} C(p) + -1^{(4-i)} p_i e_{jkl} \wedge e_i \partial_i = e_{1234} C(p) + -1^{(4-i)} \Delta p \wedge \nabla C(p)$ &emsp; &emsp; (6)

Note that $\partial_4 C(p) = 0$ since the coordinate function for the projective plane is constant for the objects we are interested in.  This leaves us with an ideal point, which is a direction vector, as we would expect.

which gives us a gradient operator for PGA that looks like this

$\nabla_* = -e_1 \partial_1 + e_2 \partial_2 - e_3 \partial_3$

if we had a different ordering of basis indices we'd potentially have different signs here.  As we will see later, this doesn't matter since the signs will cancel.

$C(p + \Delta p) \approx \textit{I} C(p) + \Delta p \wedge \nabla_* C(p) = 0$
 
The dualized statement of D'Alembert's principle (4) becomes

$\Delta p = \lambda e_{ij4} \partial_k C(p)$

Which says that any virtual displacement consistent with the constraint must be aligned with it's (dual) gradient.

Some algebra where we use the identity $(\textit{I}A \wedge B) = \textit{I}(A \cdot B)$ and $e_ke_k = 1, k \in {1,2,3}$

$-\textit{I} C(p) = \lambda e_{ij4} e_k e_k \partial_k C(p) \wedge \nabla_* C(p)$

$-\textit{I} C(p) = \lambda \textit{I} \nabla_* C(p) \wedge \nabla_* C(p)$

$-\textit{I} C(p) = \lambda \textit{I} (\nabla_* C(p) \cdot \nabla_* C(p))$

$-C(p) = \lambda \nabla_* C(p) \cdot \nabla_* C(p)$

Solving for $\Delta p$ as before and noticing $\nabla_* \cdot \nabla_*$ will cancel any sign differences from $\nabla$

$\Delta p = \frac{-C(p)}{\nabla C(p)  \cdot \nabla C(p)} e_{ij4} \partial_k C(p)$

There we have it.  The Constraint Projection Equations of PBD for PGA.  The only difference is that we have ideal points ( directions ) represented as trivectors instead of 1-vectors as seen in the standard formulation.  This is what we would expect. 

In PGA we need to translate objects with a translational dual quaternion.

# Rigid Body Dynamics


In [ ]:
using StaticArrays, LinearAlgebra, ReferenceFrameRotations

In [18]:
Vector3d = SVector{3,Float64}
Matrix3d = SMatrix{3,3,Float64}

mutable struct RigidBodyVA
    x::SVector{3,Float64}
    R::SMatrix{3,3,Float64} # body to world transform wTb
    m::Float64
    I::SVector{3,Float64}   # bodyspace
    v::SVector{3,Float64}
    w::SVector{3,Float64}   # world space
    q::Quaternion{Float64}
end

function setrotation!(rb, R::Matrix3d)
    rb.R = R
    rb.q = dcm_to_quat(R')
end

function setrotation!(rb, q::Quaternion{Float64})
    rb.q = q
    rb.R = quat_to_dcm(q)'
end

getrotation(rb) = rb.R

RigidBodyVA(x,m=1.0,inertia=ones(3)) = RigidBodyVA(x, Matrix{Float64}(I, 3,3), m, inertia, 
                                                   zeros(3), zeros(3), Quaternion(1.0,0,0,0))


skew(w::SVector{3,T}) where T = [0.0 -w[3] w[2];
                                 w[3] 0.0  -w[1];
                                 -w[2] w[1] 0.0]

function inertia_tensor(box::SVector{3,Float64}, m::Float64)
    x,y,z = box
    (m/12.0) * Vector3d(y^2+z^2, x^2+z^2, x^2+y^2)
end


function wIw_implicit(ω₀, It, wTb, dt)
    # one step of newtons method to solve for new angular velocity = f(ω′) = I(ω′-ω)+ω′xIω′*dt = 0
    # df(ω′)/ω′ = I + (1xIω′+ω′xI)*dt
    # df(ω) = I + (ωxI - Iωx1)*dt
    w = wTb'*ω₀
    Iw = It*w
    f = cross(w,Iw)*dt 
    df = It + (skew(w)*It - skew(Iw))*dt
   
    ω′ = w - df\f
    wTb*ω′
end

function integrate!(rb::RigidBodyVA, dt::Float64)
    rb.x += rb.v*dt
    #wIw = cross(rb.w, rb.R*Diagonal(rb.I)*rb.R'*rb.w)
    wIw = wIw_implicit(rb.w, Diagonal(rb.I), getrotation(rb), dt)
    rb.w = wIw
    #rb.w += rb.R*inv(Diagonal(rb.I))*rb.R'*wIw*dt
    #rb.R += skew(rb.w)*rb.R*dt
    q2 = rb.q + 0.5*dt*rb.w*rb.q
    setrotation!(rb, q2/norm(q2))
    
    return rb
end



integrate! (generic function with 1 method)

In [ ]:
# Optional: 
# These commands tell the Julia package manager to use the exact
# set of dependencies specified in the Project.toml file in this folder. 
# That should give you a nice, reproducible environment for testing. 

using Pkg
Pkg.activate(@__DIR__)
Pkg.develop(Pkg.PackageSpec(path=dirname(@__DIR__)))
Pkg.instantiate()

In [ ]:
using MeshCat, GeometryBasics, CoordinateTransformations

In [ ]:
vis = Visualizer()

In [9]:
render(vis)

MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String,MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("box1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x62, 0x6f, 0x78, 0x31], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()))))), Set{Any}(), ip"127.0.0.1", 8700))

In [ ]:
box_radius = Vec(0.05,0.2,0.4)

setobject!(vis[:box1], Rect(-box_radius, box_radius*2))

In [19]:
anim = Animation()

rb = RigidBodyVA(zeros(3))
rb.x = [0,0,0]
setrotation!(rb, Matrix3d(I))
#[0.866  0  0.5
       # 0      1  0
       # -0.5      0  0.866]
rb.w = [0.00,10,0.01]
rb.v = [0.0,-0.1,0.0]
rb.m = 10
rb.I = inertia_tensor(Vector3d(2*box_radius), rb.m)

atframe(anim, 0) do
    # within the context of atframe, calls to 
    # `settransform!` and `setprop!` are intercepted
    # and recorded in `anim` instead of having any
    # effect on `vis`.
    settransform!(vis[:box1], compose(Translation(rb.x...), LinearMap(rb.R)))
end

dt = 0.033

for i in 1:30*24
    atframe(anim, i) do
        integrate!(rb, dt)
        settransform!(vis[:box1], compose(Translation(rb.x...), LinearMap(rb.R)))
    end
end

# `setanimation!()` actually sends the animation to the
# viewer. By default, the viewer will play the animation
# right away. To avoid that, you can also pass `play=false`. 
setanimation!(vis, anim)